In [1]:
#IMPORTS

import json
import requests
import numpy as np
from tqdm import trange

#NLPAUG IMPORTS
import nlpaug.augmenter.char as char_aug
import nlpaug.augmenter.word as naw

In [2]:
#LOAD DATASET
with open("dataset.json", "r") as dataset_file:
   dataset = json.load(dataset_file)

In [3]:
#INITIALIZE AUGMENTERS
NUMBER_OF_AUG = 5

dataset_aug = {}
key_aug = char_aug.KeyboardAug(lang="fr",aug_char_min=1, aug_char_max=2,aug_word_min=1,aug_word_max=2,model_path="keyboard/fr.json")
word_aug = naw.RandomWordAug(action="swap")

In [7]:
#DATA AUGMENTATION
for i in trange(len(dataset)):
    dataset_aug[i] = dataset[i]
    sentence = dataset[i].get("sentence")
    keyboard_aug = key_aug.augment(sentence, n= NUMBER_OF_AUG) #param int n: Default is 1. Number of unique augmented output.
    wordswap_aug = augmented_text_word = word_aug.augment(sentence, n= NUMBER_OF_AUG)

    dataset_aug[i]["keyboard_aug"] = keyboard_aug
    dataset_aug[i]["wordswap_aug"] = wordswap_aug



  0%|          | 0/5395 [00:00<?, ?it/s]

  0%|          | 1/5395 [00:00<18:51,  4.77it/s]

  0%|          | 2/5395 [00:00<18:45,  4.79it/s]

  0%|          | 3/5395 [00:00<18:44,  4.79it/s]

  0%|          | 4/5395 [00:00<18:50,  4.77it/s]

  0%|          | 5/5395 [00:01<18:48,  4.78it/s]

  0%|          | 6/5395 [00:01<18:49,  4.77it/s]

  0%|          | 7/5395 [00:01<18:46,  4.78it/s]

  0%|          | 8/5395 [00:01<18:38,  4.82it/s]

  0%|          | 9/5395 [00:01<18:37,  4.82it/s]

  0%|          | 10/5395 [00:02<18:39,  4.81it/s]

  0%|          | 11/5395 [00:02<18:37,  4.82it/s]

  0%|          | 12/5395 [00:02<18:37,  4.82it/s]

  0%|          | 13/5395 [00:02<18:40,  4.80it/s]

  0%|          | 14/5395 [00:02<18:44,  4.79it/s]

  0%|          | 15/5395 [00:03<18:39,  4.80it/s]

  0%|          | 16/5395 [00:03<18:41,  4.80it/s]

  0%|          | 17/5395 [00:03<18:51,  4.75it/s]

  0%|          | 18/5395 [00:03<18:50,  4.76it/s]

  0%|          | 19/5395 [00:03<18:54,  4.74it/

  3%|▎         | 143/5395 [00:29<18:13,  4.80it/s]

  3%|▎         | 144/5395 [00:30<18:22,  4.76it/s]

  3%|▎         | 145/5395 [00:30<18:21,  4.77it/s]

  3%|▎         | 146/5395 [00:30<18:26,  4.74it/s]

  3%|▎         | 147/5395 [00:30<18:27,  4.74it/s]

  3%|▎         | 148/5395 [00:31<18:22,  4.76it/s]

  3%|▎         | 149/5395 [00:31<18:28,  4.73it/s]

  3%|▎         | 150/5395 [00:31<18:23,  4.76it/s]

  3%|▎         | 151/5395 [00:31<18:17,  4.78it/s]

  3%|▎         | 152/5395 [00:31<18:21,  4.76it/s]

  3%|▎         | 153/5395 [00:32<18:20,  4.77it/s]

  3%|▎         | 154/5395 [00:32<18:22,  4.75it/s]

  3%|▎         | 155/5395 [00:32<18:23,  4.75it/s]

  3%|▎         | 156/5395 [00:32<18:19,  4.76it/s]

  3%|▎         | 157/5395 [00:32<18:24,  4.74it/s]

  3%|▎         | 158/5395 [00:33<18:29,  4.72it/s]

  3%|▎         | 159/5395 [00:33<18:26,  4.73it/s]

  3%|▎         | 160/5395 [00:33<18:21,  4.75it/s]

  3%|▎         | 161/5395 [00:33<18:20,  4.76it/s]

  3%|▎      

  5%|▌         | 284/5395 [00:59<18:04,  4.71it/s]

  5%|▌         | 285/5395 [01:00<18:01,  4.73it/s]

  5%|▌         | 286/5395 [01:00<17:54,  4.76it/s]

  5%|▌         | 287/5395 [01:00<17:58,  4.74it/s]

  5%|▌         | 288/5395 [01:00<17:58,  4.73it/s]

  5%|▌         | 289/5395 [01:00<17:59,  4.73it/s]

  5%|▌         | 290/5395 [01:01<17:57,  4.74it/s]

  5%|▌         | 291/5395 [01:01<17:48,  4.78it/s]

  5%|▌         | 292/5395 [01:01<17:48,  4.77it/s]

  5%|▌         | 293/5395 [01:01<17:46,  4.79it/s]

  5%|▌         | 294/5395 [01:01<17:49,  4.77it/s]

  5%|▌         | 295/5395 [01:02<17:46,  4.78it/s]

  5%|▌         | 296/5395 [01:02<17:40,  4.81it/s]

  6%|▌         | 297/5395 [01:02<17:40,  4.80it/s]

  6%|▌         | 298/5395 [01:02<17:47,  4.78it/s]

  6%|▌         | 299/5395 [01:03<17:48,  4.77it/s]

  6%|▌         | 300/5395 [01:03<17:51,  4.75it/s]

  6%|▌         | 301/5395 [01:03<17:47,  4.77it/s]

  6%|▌         | 302/5395 [01:03<17:48,  4.77it/s]

  6%|▌      

  8%|▊         | 425/5395 [01:29<17:40,  4.69it/s]

  8%|▊         | 426/5395 [01:29<17:34,  4.71it/s]

  8%|▊         | 427/5395 [01:30<17:33,  4.72it/s]

  8%|▊         | 428/5395 [01:30<17:35,  4.71it/s]

  8%|▊         | 429/5395 [01:30<17:29,  4.73it/s]

  8%|▊         | 430/5395 [01:30<17:28,  4.73it/s]

  8%|▊         | 431/5395 [01:30<17:27,  4.74it/s]

  8%|▊         | 432/5395 [01:31<17:22,  4.76it/s]

  8%|▊         | 433/5395 [01:31<17:29,  4.73it/s]

  8%|▊         | 434/5395 [01:31<17:23,  4.75it/s]

  8%|▊         | 435/5395 [01:31<17:19,  4.77it/s]

  8%|▊         | 436/5395 [01:31<17:26,  4.74it/s]

  8%|▊         | 437/5395 [01:32<17:18,  4.77it/s]

  8%|▊         | 438/5395 [01:32<17:14,  4.79it/s]

  8%|▊         | 439/5395 [01:32<17:13,  4.79it/s]

  8%|▊         | 440/5395 [01:32<17:18,  4.77it/s]

  8%|▊         | 441/5395 [01:33<17:21,  4.76it/s]

  8%|▊         | 442/5395 [01:33<17:30,  4.71it/s]

  8%|▊         | 443/5395 [01:33<17:25,  4.74it/s]

  8%|▊      

 10%|█         | 566/5395 [01:59<17:07,  4.70it/s]

 11%|█         | 567/5395 [01:59<17:09,  4.69it/s]

 11%|█         | 568/5395 [01:59<17:03,  4.72it/s]

 11%|█         | 569/5395 [02:00<17:00,  4.73it/s]

 11%|█         | 570/5395 [02:00<16:55,  4.75it/s]

 11%|█         | 571/5395 [02:00<16:57,  4.74it/s]

 11%|█         | 572/5395 [02:00<16:57,  4.74it/s]

 11%|█         | 573/5395 [02:00<17:00,  4.73it/s]

 11%|█         | 574/5395 [02:01<17:01,  4.72it/s]

 11%|█         | 575/5395 [02:01<17:05,  4.70it/s]

 11%|█         | 576/5395 [02:01<17:06,  4.69it/s]

 11%|█         | 577/5395 [02:01<17:01,  4.72it/s]

 11%|█         | 578/5395 [02:01<17:00,  4.72it/s]

 11%|█         | 579/5395 [02:02<17:03,  4.70it/s]

 11%|█         | 580/5395 [02:02<17:03,  4.70it/s]

 11%|█         | 581/5395 [02:02<17:05,  4.69it/s]

 11%|█         | 582/5395 [02:02<17:07,  4.68it/s]

 11%|█         | 583/5395 [02:03<17:03,  4.70it/s]

 11%|█         | 584/5395 [02:03<17:06,  4.69it/s]

 11%|█      

...

 87%|████████▋ | 4716/5395 [16:32<02:21,  4.79it/s]

 87%|████████▋ | 4717/5395 [16:32<02:22,  4.77it/s]

 87%|████████▋ | 4718/5395 [16:33<02:21,  4.79it/s]

 87%|████████▋ | 4719/5395 [16:33<02:21,  4.76it/s]

 87%|████████▋ | 4720/5395 [16:33<02:21,  4.77it/s]

 88%|████████▊ | 4721/5395 [16:33<02:22,  4.74it/s]

 88%|████████▊ | 4722/5395 [16:33<02:21,  4.75it/s]

 88%|████████▊ | 4723/5395 [16:34<02:21,  4.74it/s]

 88%|████████▊ | 4724/5395 [16:34<02:21,  4.74it/s]

 88%|████████▊ | 4725/5395 [16:34<02:21,  4.73it/s]

 88%|████████▊ | 4726/5395 [16:34<02:20,  4.78it/s]

 88%|████████▊ | 4727/5395 [16:35<02:20,  4.76it/s]

 88%|████████▊ | 4728/5395 [16:35<02:20,  4.73it/s]

 88%|████████▊ | 4729/5395 [16:35<02:20,  4.76it/s]

 88%|████████▊ | 4730/5395 [16:35<02:20,  4.74it/s]

 88%|████████▊ | 4731/5395 [16:35<02:19,  4.75it/s]

 88%|████████▊ | 4732/5395 [16:36<02:18,  4.78it/s]

 88%|████████▊ | 4733/5395 [16:36<02:18,  4.80it/s]

 88%|████████▊ | 4734/5395 [16:36<02:17,  4.80

 90%|████████▉ | 4854/5395 [17:01<01:54,  4.74it/s]

 90%|████████▉ | 4855/5395 [17:02<01:53,  4.75it/s]

 90%|█████████ | 4856/5395 [17:02<01:53,  4.76it/s]

 90%|█████████ | 4857/5395 [17:02<01:52,  4.77it/s]

 90%|█████████ | 4858/5395 [17:02<01:52,  4.76it/s]

 90%|█████████ | 4859/5395 [17:02<01:53,  4.73it/s]

 90%|█████████ | 4860/5395 [17:03<01:52,  4.77it/s]

 90%|█████████ | 4861/5395 [17:03<01:52,  4.76it/s]

 90%|█████████ | 4862/5395 [17:03<01:52,  4.76it/s]

 90%|█████████ | 4863/5395 [17:03<01:52,  4.75it/s]

 90%|█████████ | 4864/5395 [17:04<01:51,  4.76it/s]

 90%|█████████ | 4865/5395 [17:04<01:51,  4.75it/s]

 90%|█████████ | 4866/5395 [17:04<01:51,  4.76it/s]

 90%|█████████ | 4867/5395 [17:04<01:50,  4.77it/s]

 90%|█████████ | 4868/5395 [17:04<01:50,  4.76it/s]

 90%|█████████ | 4869/5395 [17:05<01:50,  4.77it/s]

 90%|█████████ | 4870/5395 [17:05<01:50,  4.76it/s]

 90%|█████████ | 4871/5395 [17:05<01:49,  4.78it/s]

 90%|█████████ | 4872/5395 [17:05<01:49,  4.79

 93%|█████████▎| 4992/5395 [17:30<01:25,  4.73it/s]

 93%|█████████▎| 4993/5395 [17:31<01:24,  4.75it/s]

 93%|█████████▎| 4994/5395 [17:31<01:24,  4.74it/s]

 93%|█████████▎| 4995/5395 [17:31<01:24,  4.75it/s]

 93%|█████████▎| 4996/5395 [17:31<01:24,  4.73it/s]

 93%|█████████▎| 4997/5395 [17:32<01:24,  4.73it/s]

 93%|█████████▎| 4998/5395 [17:32<01:24,  4.72it/s]

 93%|█████████▎| 4999/5395 [17:32<01:23,  4.75it/s]

 93%|█████████▎| 5000/5395 [17:32<01:23,  4.73it/s]

 93%|█████████▎| 5001/5395 [17:32<01:23,  4.73it/s]

 93%|█████████▎| 5002/5395 [17:33<01:23,  4.73it/s]

 93%|█████████▎| 5003/5395 [17:33<01:22,  4.73it/s]

 93%|█████████▎| 5004/5395 [17:33<01:21,  4.77it/s]

 93%|█████████▎| 5005/5395 [17:33<01:22,  4.75it/s]

 93%|█████████▎| 5006/5395 [17:33<01:21,  4.76it/s]

 93%|█████████▎| 5007/5395 [17:34<01:21,  4.75it/s]

 93%|█████████▎| 5008/5395 [17:34<01:20,  4.79it/s]

 93%|█████████▎| 5009/5395 [17:34<01:20,  4.79it/s]

 93%|█████████▎| 5010/5395 [17:34<01:20,  4.79

 95%|█████████▌| 5130/5395 [18:00<00:55,  4.79it/s]

 95%|█████████▌| 5131/5395 [18:00<00:54,  4.81it/s]

 95%|█████████▌| 5132/5395 [18:00<00:54,  4.80it/s]

 95%|█████████▌| 5133/5395 [18:01<00:54,  4.77it/s]

 95%|█████████▌| 5134/5395 [18:01<00:54,  4.77it/s]

 95%|█████████▌| 5135/5395 [18:01<00:54,  4.76it/s]

 95%|█████████▌| 5136/5395 [18:01<00:54,  4.74it/s]

 95%|█████████▌| 5137/5395 [18:02<00:54,  4.74it/s]

 95%|█████████▌| 5138/5395 [18:02<00:54,  4.74it/s]

 95%|█████████▌| 5139/5395 [18:02<00:54,  4.74it/s]

 95%|█████████▌| 5140/5395 [18:02<00:53,  4.75it/s]

 95%|█████████▌| 5141/5395 [18:02<00:53,  4.79it/s]

 95%|█████████▌| 5142/5395 [18:03<00:52,  4.78it/s]

 95%|█████████▌| 5143/5395 [18:03<00:53,  4.72it/s]

 95%|█████████▌| 5144/5395 [18:03<00:52,  4.76it/s]

 95%|█████████▌| 5145/5395 [18:03<00:52,  4.77it/s]

 95%|█████████▌| 5146/5395 [18:03<00:52,  4.78it/s]

 95%|█████████▌| 5147/5395 [18:04<00:51,  4.80it/s]

 95%|█████████▌| 5148/5395 [18:04<00:51,  4.79

 98%|█████████▊| 5268/5395 [18:29<00:26,  4.74it/s]

 98%|█████████▊| 5269/5395 [18:30<00:26,  4.76it/s]

 98%|█████████▊| 5270/5395 [18:30<00:26,  4.76it/s]

 98%|█████████▊| 5271/5395 [18:30<00:25,  4.77it/s]

 98%|█████████▊| 5272/5395 [18:30<00:25,  4.76it/s]

 98%|█████████▊| 5273/5395 [18:30<00:25,  4.75it/s]

 98%|█████████▊| 5274/5395 [18:31<00:25,  4.73it/s]

 98%|█████████▊| 5275/5395 [18:31<00:25,  4.70it/s]

 98%|█████████▊| 5276/5395 [18:31<00:25,  4.70it/s]

 98%|█████████▊| 5277/5395 [18:31<00:25,  4.70it/s]

 98%|█████████▊| 5278/5395 [18:31<00:24,  4.75it/s]

 98%|█████████▊| 5279/5395 [18:32<00:24,  4.77it/s]

 98%|█████████▊| 5280/5395 [18:32<00:24,  4.77it/s]

 98%|█████████▊| 5281/5395 [18:32<00:23,  4.79it/s]

 98%|█████████▊| 5282/5395 [18:32<00:23,  4.78it/s]

 98%|█████████▊| 5283/5395 [18:33<00:23,  4.78it/s]

 98%|█████████▊| 5284/5395 [18:33<00:23,  4.76it/s]

 98%|█████████▊| 5285/5395 [18:33<00:23,  4.77it/s]

 98%|█████████▊| 5286/5395 [18:33<00:22,  4.79

In [5]:
"""#DATA AUG NO DUPLICATES
for i in trange(len(dataset)):
    dataset_aug[i] = dataset[i]
    sentence = dataset[i].get("sentence")
    keyboard_aug = []
    wordswap_aug=[]
    
    while len(keyboard_aug) < NUMBER_OF_AUG:
        augmented_text_typo = key_aug.augment(sentence)
        augmented_text_word = word_aug.augment(sentence)
        
        if augmented_text_typo not in keyboard_aug:
            keyboard_aug.append(augmented_text_typo)
            
        if augmented_text_word not in wordswap_aug:
            wordswap_aug.append(augmented_text_word)
            
    dataset_aug[i]["keyboard_aug"] = keyboard_aug
    dataset_aug[i]["wordswap_aug"] = wordswap_aug"""

 32%|███▏      | 1730/5395 [30:00<1:03:34,  1.04s/it]

KeyboardInterrupt: 

In [ ]:
#SAVE AUGMENTED DATASET
with open("dataset_aug_typos_words.json", "w") as dataset_aug_file:
    json.dump(dataset, dataset_aug_file, ensure_ascii=False)